In [1]:
using Gen

# An even dumber example: a Gaussian model

An improper distribution -- i.e., it's not correctly normalized. We'll see if we can sample from it anyway.

In [2]:
struct DPMGaussian <: Gen.Distribution{Float64} end
const dpmgaussian = DPMGaussian()

Gen.random(dpmgaussian, mu, sigma) = mu
Gen.logpdf(dpmgaussian, x, mu, sigma) = -0.5*(x - mu)^2 / (sigma^2) - 0.5*log(2 * pi * sigma^2)

dpmgaussian(mu, sigma) = Gen.random(dpmgaussian, mu, sigma)

DPMGaussian()

In [10]:
struct Unspecified <: Gen.Distribution{Float64} end
const unspecified = Unspecified()

Gen.random(unspecified, default) = default
Gen.logpdf(unspecified, x, default) = 0.0

unspecified(default) = Gen.random(unspecified, default)

Unspecified()

## Write a probabilistic program

In [32]:
@gen function dpm_gaussian(mu::Float64, sigma::Float64)
    
    #a = @trace(unspecified(0.0), :a)
    
    z = @trace(dpmgaussian(mu, sigma), :z)
    return z
end

DynamicDSLFunction{Any}(Dict{Symbol,Any}(), Dict{Symbol,Any}(), Type[Float64, Float64], ##dpm_gaussian#376, Bool[0, 0], false)

## Inference: Metropolis-Hastings sampling

Define a proposal distribution

In [33]:
@gen function dpmg_proposal(cur_tr)
    a = 0.5*@trace(uniform_discrete(0,1), :a)
    z = @trace(bernoulli(a), :z)
    return z
end

DynamicDSLFunction{Any}(Dict{Symbol,Any}(), Dict{Symbol,Any}(), Type[Any], ##dpmg_proposal#377, Bool[0], false)

In [35]:
tr, _ = Gen.generate(dpm_gaussian, (0.0, 1.0))
#println("a = ", tr[:a], ";\tz = ", tr[:z])

(Gen.DynamicDSLTrace{DynamicDSLFunction{Any}}(DynamicDSLFunction{Any}(Dict{Symbol,Any}(), Dict{Symbol,Any}(), Type[Float64, Float64], ##dpm_gaussian#376, Bool[0, 0], false), Trie{Any,Gen.ChoiceOrCallRecord}(Dict{Any,Gen.ChoiceOrCallRecord}(:z => Gen.ChoiceOrCallRecord{Float64}(0.0, -0.9189385332046727, NaN, true)), Dict{Any,Trie{Any,Gen.ChoiceOrCallRecord}}()), false, -0.9189385332046727, 0.0, (0.0, 1.0), 0.0), 0.0)

In [36]:
new_tr, fing1, fing2 = Gen.propose(dpmg_proposal, (tr,))
#println("a = ", new_tr[:a], ";\tz = ", new_tr[:z])

(DynamicChoiceMap(Dict{Any,Any}(:a => 0,:z => false), Dict{Any,Any}()), -0.6931471805599453, false)

In [29]:
cm = Gen.choicemap()
cm[:z] = new_tr[:z]
#cm[:a] = new_tr[:a]
w, r = Gen.assess(dpmg_proposal, (tr,), cm)

KeyError: KeyError: key :a not found

In [28]:
print(log.([0.25; 0.5; 0.75]))

[-1.3862943611198906, -0.6931471805599453, -0.2876820724517809]